<a href="https://colab.research.google.com/github/feiyoung/ReadPapers/blob/master/DGL_learn_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dgl
#import dgl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 17.4 MB/s eta 0:00:00


In [2]:
import dgl

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [3]:
import torch as th

In [10]:
#u, v = th.tensor([0, 0, 0, 1]), th.tensor([1, 2, 3, 3])
## Define edge using two groups of node IDs.
u, v = th.tensor([0,0,0,1]), th.tensor([1,2,3,4])

In [5]:
u

tensor([0, 0, 0, 1])

In [11]:
g = dgl.graph((u, v))

In [12]:
print(g)

Graph(num_nodes=5, num_edges=4,
      ndata_schemes={}
      edata_schemes={})


In [13]:
# Access the node IDs in the graph
print(g.nodes())

tensor([0, 1, 2, 3, 4])


In [14]:
# Access the node ID of edges
print(g.edges())

(tensor([0, 0, 0, 1]), tensor([1, 2, 3, 4]))


In [15]:
# Access all the node IDs of edges and the IDs of edges
print(g.edges(form='all'))

(tensor([0, 0, 0, 1]), tensor([1, 2, 3, 4]), tensor([0, 1, 2, 3]))


In [16]:
# 如果具有最大ID的节点没有边，在创建图的时候，用户需要明确地指明节点的数量。
g = dgl.graph((u, v), num_nodes=8)

In [17]:
print(g)

Graph(num_nodes=8, num_edges=4,
      ndata_schemes={}
      edata_schemes={})


In [18]:
print(g.edges(form='all'))

(tensor([0, 0, 0, 1]), tensor([1, 2, 3, 4]), tensor([0, 1, 2, 3]))


In [19]:
print(g.nodes()) # Now there are 8 nodes!!

tensor([0, 1, 2, 3, 4, 5, 6, 7])


In [20]:
# 对于无向的图，用户需要为每条边都创建两个方向的边。可以使用 dgl.to_bidirected() 函数来实现这个目的。
# 如下面的代码段所示，这个函数可以把原图转换成一个包含反向边的图。
bg = dgl.to_bidirected(g)

In [21]:
print(bg) # Now there are 8 edges

Graph(num_nodes=8, num_edges=8,
      ndata_schemes={}
      edata_schemes={})


In [22]:
print(bg.edges())

(tensor([0, 0, 0, 1, 1, 2, 3, 4]), tensor([1, 2, 3, 0, 4, 0, 0, 1]))


Note: 由于Tensor类内部使用C来存储，且显性定义了数据类型以及存储的设备信息，DGL推荐使用Tensor作为DGL API的输入。
DGL支持使用 32
 位或 64
 位的整数作为节点ID和边ID。节点和边ID的数据类型必须一致。如果使用 64
 位整数， DGL可以处理最多 263−1
 个节点或边。不过，如果图里的节点或者边的数量小于 231−1
 ，用户最好使用 32
 位整数。 这样不仅能提升速度，还能减少内存的使用。DGL提供了进行数据类型转换的方法，如下例所示。

In [24]:
edges = th.tensor([2, 5, 3]), th.tensor([3, 5, 0])  # 边：2->3, 5->5, 3->0
g64 = dgl.graph(edges)
print(g64.idtype)

torch.int64


In [25]:
g32 = dgl.graph(edges, idtype=th.int32)  # 使用int32构建图
print(g32.idtype)

torch.int32


In [26]:
g64_2 = g32.long()  # 转换成int64
g64_2.idtype

torch.int64

In [27]:
g32_2 = g64.int()  # 转换成int32
g32_2.idtype

torch.int32

Create attributes for nodes and edges of Graph

In [28]:
print(g.num_nodes())

8


In [29]:
g.ndata['x'] = th.ones(g.num_nodes(), 3) # each node have a vector [1,1,1] and the attribute name is 'x'

In [30]:
g.edata['x'] = th.ones(g.num_edges(), dtype=th.int32)  # each edge has a scalar 1 and integer type, and the attribute name is 'x'

In [31]:
print(g)

Graph(num_nodes=8, num_edges=4,
      ndata_schemes={'x': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={'x': Scheme(shape=(), dtype=torch.int32)})


In [34]:
g.ndata['x'][2] #  # 获取节点2的特征

tensor([1., 1., 1.])

In [32]:
g.ndata['y'] = th.randn(g.num_nodes(), 5) ## Add a new group of attributes named 'y' for nodes.

In [35]:
g.edata['x'][th.tensor([0, 3])]  # 获取边0和3的特征

tensor([1, 1], dtype=torch.int32)

关于 ndata 和 edata 接口的重要说明：仅允许使用数值类型（如单精度浮点型、双精度浮点型和整型）的特征。这些特征可以是标量、向量或多维张量。

每个节点特征具有唯一名称，每个边特征也具有唯一名称。节点和边的特征可以具有相同的名称（如上述示例代码中的 'x' ）。

通过张量分配创建特征时，DGL会将特征赋给图中的每个节点和每条边。该张量的第一维必须与图中节点或边的数量一致。 不能将特征赋给图中节点或边的子集。

相同名称的特征必须具有相同的维度和数据类型。

特征张量使用”行优先”的原则，即每个行切片储存1个节点或1条边的特征（参考上述示例代码的第16和18行）。

对于加权图，用户可以将权重储存为一个边特征，如下。

In [36]:
# 边 0->1, 0->2, 0->3, 1->3
edges = th.tensor([0, 0, 0, 1]), th.tensor([1, 2, 3, 3])
weights = th.tensor([0.1, 0.6, 0.9, 0.7])  # 每条边的权重
g = dgl.graph(edges)
g.edata['w'] = weights  # 将其命名为 'w'
print(g)


Graph(num_nodes=4, num_edges=4,
      ndata_schemes={}
      edata_schemes={'w': Scheme(shape=(), dtype=torch.float32)})


1.4 从外部源创建图¶

In [38]:
#可以从外部来源构造一个 DGLGraph 对象，包括：
# 1.从用于图和稀疏矩阵的外部Python库（NetworkX 和 SciPy）创建而来。
# 2. 从磁盘加载图数据。
import dgl
import torch as th
import scipy.sparse as sp
spmat = sp.rand(100, 100, density=0.05) # 5%非零项
g2 = dgl.from_scipy(spmat)
print(g2)

Graph(num_nodes=100, num_edges=500,
      ndata_schemes={}
      edata_schemes={})


In [39]:
print(g2.nodes())

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
        72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
        90, 91, 92, 93, 94, 95, 96, 97, 98, 99])


In [40]:
print(g2.edges(form='all'))

(tensor([45,  7, 13, 10, 25, 16, 95, 99, 71, 78, 97, 53, 42, 47, 15, 16, 23, 32,
        48, 32, 31, 83, 58, 25, 42, 67, 22, 44, 31, 24,  9, 14, 69, 37, 66, 39,
         2, 34, 35, 90, 74,  5, 45, 31, 26, 80, 96, 55, 82, 39, 11, 17, 14, 62,
        52, 44, 16, 38, 96,  9, 72, 33, 49, 99, 14,  2, 89, 12, 15,  3, 84, 66,
        28, 67, 92, 38, 71, 56, 86, 27, 93, 95, 22, 25, 44, 67,  5, 70, 69, 21,
        32, 21, 10, 52, 99, 25, 68, 61, 44, 25, 29, 15, 64, 20, 69, 52, 77, 20,
        60, 94, 24, 75, 38, 94,  6, 67, 46, 14, 43, 53, 79, 88, 17, 74, 50, 64,
        73,  3, 75, 79, 30, 53, 12, 13, 39, 62, 66, 41, 34, 12, 20, 65, 89, 28,
        38,  9, 82, 85, 53, 11, 67, 95, 19, 22, 23, 71,  4, 27, 34, 36,  6, 16,
        93, 79, 82, 75, 91, 39, 12, 70, 40, 86, 24, 42, 70, 38, 50, 47, 60, 16,
        48, 30, 96, 82, 24, 12, 93, 96,  0, 81, 74, 10, 48, 36, 58, 27, 56, 19,
        45, 43,  5, 88, 97,  5, 20,  4, 17, 88, 27, 56, 41, 21, 47, 12, 23, 13,
        56, 24, 84, 87, 43, 32,  5,  7,

In [41]:
import networkx as nx
nx_g = nx.path_graph(5) # 一条链路0-1-2-3-4
dgl.from_networkx(nx_g) #

Graph(num_nodes=5, num_edges=8,
      ndata_schemes={}
      edata_schemes={})

Note: 注意，当使用 nx.path_graph(5) 进行创建时， DGLGraph 对象有8条边，而非4条。 这是由于 nx.path_graph(5) 构建了一个无向的NetworkX图 networkx.Graph ，而 DGLGraph 的边总是有向的。 所以当将无向的NetworkX图转换为 DGLGraph 对象时，DGL会在内部将1条无向边转换为2条有向边。 使用有向的NetworkX图 networkx.DiGraph 可避免该行为。

In [42]:
nxg = nx.DiGraph([(2, 1), (1, 2), (2, 3), (0, 0)])
dgl.from_networkx(nxg)

Graph(num_nodes=4, num_edges=4,
      ndata_schemes={}
      edata_schemes={})

从磁盘加载图¶
有多种文件格式可储存图，所以这里难以枚举所有选项。本节仅给出一些常见格式的一般情况。

逗号分隔值（CSV）.CSV是一种常见的格式，以表格格式储存节点、边及其特征：

1.5 创建异构图

In [43]:
import dgl
import torch as th

# 创建一个具有3种节点类型和3种边类型的异构图
graph_data = {
   ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 2])),
   ('drug', 'interacts', 'gene'): (th.tensor([0, 1]), th.tensor([2, 3])),
   ('drug', 'treats', 'disease'): (th.tensor([1]), th.tensor([2]))
}
g = dgl.heterograph(graph_data)
g.ntypes

['disease', 'drug', 'gene']

In [44]:
print(g)

Graph(num_nodes={'disease': 3, 'drug': 3, 'gene': 4},
      num_edges={('drug', 'interacts', 'drug'): 2, ('drug', 'interacts', 'gene'): 2, ('drug', 'treats', 'disease'): 1},
      metagraph=[('drug', 'drug', 'interacts'), ('drug', 'gene', 'interacts'), ('drug', 'disease', 'treats')])


In [45]:
print(g.etypes) # print edge types

['interacts', 'interacts', 'treats']


In [46]:
print(g.canonical_etypes) # output the canonical edge types.

[('drug', 'interacts', 'drug'), ('drug', 'interacts', 'gene'), ('drug', 'treats', 'disease')]
